In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.optimize as sco

# Define objectives and constraints
investment_horizon = 5  # years
risk_tolerance = 'medium'  # risk tolerance level
risk_free_rate = 0.01  # 1% annual risk-free rate

# List of stock tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']

# Download historical price data
data = yf.download(tickers, start="2018-01-01", end="2023-01-01")
prices = data['Adj Close']

# Calculate daily returns
returns = prices.pct_change().dropna()

# Calculate expected annual returns
expected_returns = returns.mean() * 252

# Calculate annualized covariance matrix
cov_matrix = returns.cov() * 252

# Function to calculate portfolio statistics
def portfolio_stats(weights, returns, cov_matrix):
    port_return = np.sum(weights * returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return port_return, port_volatility

# Function to minimize (negative Sharpe ratio)
def min_sharpe(weights, returns, cov_matrix, risk_free_rate):
    port_return, port_volatility = portfolio_stats(weights, returns, cov_matrix)
    return -((port_return - risk_free_rate) / port_volatility)

# Constraints and bounds
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # weights sum to 1
bounds = tuple((0, 1) for _ in range(len(tickers)))  # weights between 0 and 1

# Initial guess
init_guess = len(tickers) * [1. / len(tickers)]

# Optimize for maximum Sharpe ratio
opt_results = sco.minimize(min_sharpe, init_guess, args=(expected_returns, cov_matrix, risk_free_rate), method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
opt_weights = opt_results.x

# Calculate optimized portfolio performance
opt_return, opt_volatility = portfolio_stats(opt_weights, expected_returns, cov_matrix)
sharpe_ratio = (opt_return - risk_free_rate) / opt_volatility

print(f"Optimized Portfolio Weights: {opt_weights}")
print(f"Optimized Portfolio Return: {opt_return}")
print(f"Optimized Portfolio Volatility: {opt_volatility}")
print(f"Optimized Portfolio Sharpe Ratio: {sharpe_ratio}")


[*********************100%***********************]  5 of 5 completed

Optimized Portfolio Weights: [4.90702523e-01 0.00000000e+00 0.00000000e+00 2.42861287e-16
 5.09297477e-01]
Optimized Portfolio Return: 0.2764475184251076
Optimized Portfolio Volatility: 0.30352259380383834
Optimized Portfolio Sharpe Ratio: 0.8778506900784733
